![title](https://www.roomcloud.net/wordpress/wp-content/uploads/2019/05/Progetto-senza-titolo-1.jpg)
#  <span style="color:orange"><center>**Kayak**</center></span> 

L'équipe marketing a besoin d'aide pour un nouveau projet. Après avoir effectué des recherches sur les utilisateurs, l'équipe a découvert que 70 % de leurs utilisateurs qui planifient un voyage, aimeraient avoir plus d'informations sur la destination vers laquelle ils se rendent .

les clients ont tendance à se méfier des informations qu'ils lisent s'ils ne connaissent pas la marque qui a produit le contenu.

Par conséquent, Kayak Marketing Team souhaite créer une application qui recommandera aux gens où planifier leurs prochaines vacances.

L'équipe marketing souhaite se concentrer d'abord sur les meilleures villes où voyager en France. 

In [1]:
!pip install plotly -q

In [2]:
import requests
import json 
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os 
import logging
from datetime import datetime
from bs4 import BeautifulSoup
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "iframe"

## API scraping

In [3]:
#liste des 35 meilleures villes  selectionnée selon One Week In.com
liste_ville = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens",
               "Lille","Strasbourg","Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon",
               "Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon","Bormes les Mimosas","Cassis","Marseille",
                "Aix en Provence","Avignon","Uzes",
                "Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne",
                "Ariege","Toulouse","Montauban","Biarritz","Bayonne","La Rochelle"]

In [4]:
# liste de la longitude et la latitude des 35 villes selectionnées 
#grâce à la librairie requests
liste_gps = []

for element in liste_ville:

    #API
    r_gps = requests.get("https://nominatim.openstreetmap.org/search?q={}&format=json".format(element)) 
    dic_gps = r_gps.json()
    latitude = float(dic_gps[0]["lat"])
    longitude = float(dic_gps[0]["lon"])
    liste_gps.append([latitude,longitude])
    

In [5]:
#on crée un ID pour ajouter au datafram et on fusionne les datafram plus tard
ID = [i+1 for i in range(len(liste_ville))]

In [6]:
# on crée un datafram contenant les 35 villes et la géolocalisation ainsi que leurs ID
ID = [i for i in range(len(liste_ville))]
dataset = pd.DataFrame(liste_gps)
dataset['ville']= liste_ville
dataset ['ID']=ID
dataset.columns = ['longitude','latitude', 'villes','ID']
#display(dataset)

In [7]:
# on recupère la météo des 35 villes selectionnées sur 7 jours
liste_meteo=[]

for x in range(len(liste_ville)):
    ville = liste_ville[x]
    Id= x
    latitude1 = liste_gps[x][0]
    longitude2 = liste_gps[x][1]

    r_meteo = requests.get("https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&units=metric&lang=fr&exclude=minutely,hourly&appid=deed******bfe*bc****c****e*****d&format=json".format(latitude1,longitude2))
    dic_meteo = r_meteo.json()
   
    for méteo in range(1,8):
        lat=float(dic_meteo['lat'])
        lon=float(dic_meteo['lon'])
        date_scrap = dic_meteo['daily'][méteo]['dt']
        date = datetime.fromtimestamp (date_scrap)
        dates = date.strftime("%d/%m/%y")
        temps = dic_meteo["daily"][méteo]["weather"][0]["description"]
        temperature_max = float (dic_meteo["daily"][méteo]['temp']['max'])
        temperature_min = float (dic_meteo["daily"][méteo]['temp']['min'])
        temperature_jour = float(dic_meteo["daily"][méteo]['temp']['day'])
        liste_meteo.append([lat,lon,ville,Id,dates,temps,temperature_max,temperature_min,temperature_jour])
        

In [8]:
dataset2= pd.DataFrame(liste_meteo)
dataset2.columns = ['lat','lon','Nom_Ville','Id','dates','temps','temperature_max','temperature_min','temperature_jour']
#dataset2

In [9]:
# je fais une analyse des villes les plus chaudes en calculant la moyenne 
#des températures des plus chaudes sur 7 jours qui est mon critère pour définir les meilleures vacances
avg_tmax = dataset2.groupby(['Nom_Ville'])['temperature_max'].mean()
avg_tmax = avg_tmax.sort_values(ascending = False)
avg_tmax = pd.DataFrame(avg_tmax)
fusion = pd.merge(dataset, avg_tmax, how="right", left_on="villes", right_on="Nom_Ville")
fusion

,longitude,latitude,villes,ID,temperature_max
0,43.837425,4.360069,Nimes,24,29.431429
1,43.949249,4.805901,Avignon,22,29.062857
2,44.012128,4.419672,Uzes,23,28.840000
3,43.529842,5.447474,Aix en Provence,21,28.198571
4,43.150697,6.341928,Bormes les Mimosas,18,26.608571
5,43.565823,4.191284,Aigues Mortes,25,26.371429
6,42.525050,3.083155,Collioure,27,25.890000
7,43.213036,2.349107,Carcassonne,28,25.782857
8,43.214036,5.539632,Cassis,19,25.717143
9,43.296174,5.369953,Marseille,20,25.622857


## URL scrapping

In [10]:
#on récupère 20 hotels ainsi que leurs informations en fonction de nos 35 villes selectionnées précédemment 
#grâce a beautifulsoup
liste_booking=[]

#soup = BeautifulSoup("<html>data</html>", "html.parser")

for hotels in range(len(liste_ville)):
    villes = liste_ville[hotels]
    Id= hotels
       
    r_booking = requests.get("https://www.booking.com/searchresults.fr.html?ss={}".format(liste_ville[hotels]), headers = {'User-Agent': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'})
    soup = BeautifulSoup(r_booking.content)
    link=soup.select("div.b_hotelSummary")

    adresse_hotel = [element.get_text().replace('\n','') for element in soup.find_all("p","b_hotelAddress")]
    hotel_description = [element.get_text().replace('\n','') for element in soup.find_all("p","hotel_description")]
    hotels_url = [x.get('href') for x in soup.select("h3 a")] 
    nom_hotel = [x.get('title') for x in soup.select('div.hotel_information a img[title]')]
    hotels_score =[x.get('alt') if x.get('alt') != "" else '0' for x in soup.select('div.hotel_review_score b img[alt]')]
    for explode in range(len(link)):
        data = {
        "ID":Id,
        "villes":villes,
        "name_hotels":nom_hotel[explode], 
        "adresse_hotel":adresse_hotel[explode],
        "hotels_url":hotels_url[explode],
        "hotel_description":hotel_description[explode],
        "hotels_score":hotels_score
    }
    
        liste_booking.append(data)

data=pd.DataFrame(liste_booking)
    
        #data["hotels_score"] = data["hotels_score"].apply(lambda x: if = x.append(np.nan))
    
    #for row in data.iterrows:
        #print(row["ID"], row["villes"],row["name_hotels"], row["adresse_hotel"],row["hotels_url"], row["hotel_description"],row["hotels_score"])

In [11]:
data

,ID,villes,name_hotels,adresse_hotel,hotels_url,hotel_description,hotels_score
0,0,Mont Saint Michel,Hôtel Vert,"La Caserne, Le Mont-Saint-Michel",/hotel/fr/vert.fr.html?label=gen173nr-1FCAQogg...,"Situé à 2 km du Mont-Saint-Michel, sur la côte...","[8.2/10, 8.2/10, 7.5/10, 8.2/10, 7.6/10, 7.3/1..."
1,0,Mont Saint Michel,Mercure Mont Saint Michel,"La Caserne, Le Mont-Saint-Michel",/hotel/fr/mont-saint-michel.fr.html?label=gen1...,Installé dans des espaces verts à seulement 2 ...,"[8.2/10, 8.2/10, 7.5/10, 8.2/10, 7.6/10, 7.3/1..."
2,0,Mont Saint Michel,La Vieille Auberge,"Grande Rue, Le Mont-Saint-Michel",/hotel/fr/la-vieille-auberge-le-mont-saint-mic...,L'établissement La Vieille Auberge est situé d...,"[8.2/10, 8.2/10, 7.5/10, 8.2/10, 7.6/10, 7.3/1..."
3,0,Mont Saint Michel,Hotel Gabriel,"Route du Mont Saint Michel, Le Mont-Saint-Michel",/hotel/fr/hotel-gabriel.fr.html?label=gen173nr...,"L'Hotel Gabriel se trouve à 1,6 km du Mont-Sai...","[8.2/10, 8.2/10, 7.5/10, 8.2/10, 7.6/10, 7.3/1..."
4,0,Mont Saint Michel,Hôtel la Croix Blanche,"grande rue, Le Mont-Saint-Michel",/hotel/fr/ha-el-la-croix-blanche.fr.html?label...,Installé au cœur du village médiéval du Mont-S...,"[8.2/10, 8.2/10, 7.5/10, 8.2/10, 7.6/10, 7.3/1..."
...,...,...,...,...,...,...,...
695,34,La Rochelle,Hôtel Atlantic,"23 Rue Verdière, La Rochelle",/hotel/fr/atlantic-la-rochelle.fr.html?label=g...,L'Atlantic Hôtel se situe dans le centre de La...,"[8.4/10, 7.2/10, 7.4/10, 8.2/10, 7.5/10, 7.5/1..."
696,34,La Rochelle,Hôtel Henri IV,"31 Rue Des Gentilshommes, La Rochelle",/hotel/fr/henri-iv.fr.html?label=gen173nr-1FCA...,Situé au cœur du centre-ville historique de La...,"[8.4/10, 7.2/10, 7.4/10, 8.2/10, 7.5/10, 7.5/1..."
697,34,La Rochelle,Hôtel Saint Nicolas,"13 rue Sardinerie, La Rochelle",/hotel/fr/comforthotelsaintnicolas.fr.html?lab...,L'Hôtel Saint Nicolas dispose d'une connexion ...,"[8.4/10, 7.2/10, 7.4/10, 8.2/10, 7.5/10, 7.5/1..."
698,34,La Rochelle,Un Hotel sur le Port,"5 Rue De La Fabrique, La Rochelle",/hotel/fr/axe.fr.html?label=gen173nr-1FCAQoggJ...,"Situé au cœur de La Rochelle, à 5 minutes de l...","[8.4/10, 7.2/10, 7.4/10, 8.2/10, 7.5/10, 7.5/1..."


In [12]:
liste_url = data['hotels_url'].tolist()

In [13]:
# je récupère la géolocalisation de mes hotels grâce à l'url de chaque hotel
geo_hotel = []

for hotels in range(len(liste_url)):

       
    r_geo = requests.get("https://www.booking.com{}".format(liste_url[hotels]), headers = {'User-Agent': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'})
    soup = BeautifulSoup(r_geo.content)
    link = soup.select("a.show_map")
    
    name_hotel = [x.get('title') for x in soup.select('#b_google_map_thumbnail img')]
    geo = [x.get('data-coords') for x in soup.select("#show_map")][0]
    long = float(geo.split(",")[0])
    lati = float(geo.split(",")[1])

    geo_hotel.append([lati,long, name_hotel])

In [14]:
dataset3= pd.DataFrame(geo_hotel)
dataset3.columns=['latitude','longitude','name_hotel']
#result = pd.concat([data, dataset3], axis=1, join="inner")
result = pd.concat([data, dataset3], axis=1).reindex(data.index)
#result1 = pd.merge(villes, name_hotel, how="right", left_on="name_hotel", right_on="name_hotels")

In [15]:
result

,ID,villes,name_hotels,adresse_hotel,hotels_url,hotel_description,hotels_score,latitude,longitude,name_hotel
0,0,Mont Saint Michel,Hôtel Vert,"La Caserne, Le Mont-Saint-Michel",/hotel/fr/vert.fr.html?label=gen173nr-1FCAQogg...,"Situé à 2 km du Mont-Saint-Michel, sur la côte...","[8.2/10, 8.2/10, 7.5/10, 8.2/10, 7.6/10, 7.3/1...",48.614700,-1.509617,[Hôtel Vert]
1,0,Mont Saint Michel,Mercure Mont Saint Michel,"La Caserne, Le Mont-Saint-Michel",/hotel/fr/mont-saint-michel.fr.html?label=gen1...,Installé dans des espaces verts à seulement 2 ...,"[8.2/10, 8.2/10, 7.5/10, 8.2/10, 7.6/10, 7.3/1...",48.614247,-1.510545,[Mercure Mont Saint Michel]
2,0,Mont Saint Michel,La Vieille Auberge,"Grande Rue, Le Mont-Saint-Michel",/hotel/fr/la-vieille-auberge-le-mont-saint-mic...,L'établissement La Vieille Auberge est situé d...,"[8.2/10, 8.2/10, 7.5/10, 8.2/10, 7.6/10, 7.3/1...",48.636063,-1.511457,[La Vieille Auberge]
3,0,Mont Saint Michel,Hotel Gabriel,"Route du Mont Saint Michel, Le Mont-Saint-Michel",/hotel/fr/hotel-gabriel.fr.html?label=gen173nr...,"L'Hotel Gabriel se trouve à 1,6 km du Mont-Sai...","[8.2/10, 8.2/10, 7.5/10, 8.2/10, 7.6/10, 7.3/1...",48.615381,-1.510710,[Hotel Gabriel]
4,0,Mont Saint Michel,Hôtel la Croix Blanche,"grande rue, Le Mont-Saint-Michel",/hotel/fr/ha-el-la-croix-blanche.fr.html?label...,Installé au cœur du village médiéval du Mont-S...,"[8.2/10, 8.2/10, 7.5/10, 8.2/10, 7.6/10, 7.3/1...",48.635734,-1.509861,[Hôtel la Croix Blanche]
...,...,...,...,...,...,...,...,...,...,...
695,34,La Rochelle,Hôtel Atlantic,"23 Rue Verdière, La Rochelle",/hotel/fr/atlantic-la-rochelle.fr.html?label=g...,L'Atlantic Hôtel se situe dans le centre de La...,"[8.4/10, 7.2/10, 7.4/10, 8.2/10, 7.5/10, 7.5/1...",46.157274,-1.155219,[Hôtel Atlantic]
696,34,La Rochelle,Hôtel Henri IV,"31 Rue Des Gentilshommes, La Rochelle",/hotel/fr/henri-iv.fr.html?label=gen173nr-1FCA...,Situé au cœur du centre-ville historique de La...,"[8.4/10, 7.2/10, 7.4/10, 8.2/10, 7.5/10, 7.5/1...",46.159023,-1.151601,[Hôtel Henri IV]
697,34,La Rochelle,Hôtel Saint Nicolas,"13 rue Sardinerie, La Rochelle",/hotel/fr/comforthotelsaintnicolas.fr.html?lab...,L'Hôtel Saint Nicolas dispose d'une connexion ...,"[8.4/10, 7.2/10, 7.4/10, 8.2/10, 7.5/10, 7.5/1...",46.158135,-1.148913,[Hôtel Saint Nicolas]
698,34,La Rochelle,Un Hotel sur le Port,"5 Rue De La Fabrique, La Rochelle",/hotel/fr/axe.fr.html?label=gen173nr-1FCAQoggJ...,"Situé au cœur de La Rochelle, à 5 minutes de l...","[8.4/10, 7.2/10, 7.4/10, 8.2/10, 7.5/10, 7.5/1...",46.156121,-1.148914,[Un Hotel sur le Port]


In [17]:
#je crée une carte qui affiche la météo des 35 villes
list_column = ['villes']


fig = px.scatter_mapbox(fusion,
                        lat="longitude",
                        lon="latitude",
                        color="temperature_max",
                        size="temperature_max",
                        zoom=10,
                        #color_discrete_sequence=["red"],
                        #text='Villes',
                        title='Temperature moyenne',
                        mapbox_style="carto-positron",
                        hover_data=list_column,)
fig.show()

![title](list_meteo.png)

In [22]:
#je crée une carte qui affiche les 20 hotels des 35 villes
liste_column = ['villes', 'name_hotels']

fig = px.scatter_mapbox(result,
                        lat='latitude',
                        lon="longitude",
                        color="ID",
                        size=result["latitude"],
                        zoom=10,
                        #color_discrete_sequence=["red"],
                        #text='Villes',
                        title='liste hotel france',
                        mapbox_style="carto-positron",
                        hover_data=liste_column,)
fig.show()

![title](list_hotel.png)

In [19]:
dataset.to_csv('geolocalisation.csv')
dataset2.to_csv('meteo.csv')
fusion.to_csv('avg_meteo.csv')
result.to_csv('hotel_booking.csv',encoding='utf-8',index=False, sep="\t")

## Data lake S3 bucket

In [ ]:
!pip install boto3 -q

In [ ]:
import boto3

# configurer notre session d'utilisateur
MY_acces_KEY = "***********"
MY_secret_KEY = "**********"

session = boto3.Session(aws_access_key_id= MY_acces_KEY, 
                        aws_secret_access_key= MY_secret_KEY)

Maintenant que nous avons spécifié notre session avec les bonnes informations d'identification, nous pouvons configurer une ressource.
On crée un Bucket "Kayak-project"

In [ ]:
s3 = session.resource("s3")
bucket = s3.create_bucket(Bucket="kayak-project")

maintenant nous allons charger nos csv dans notre bucket S3

In [ ]:
import pandas as pd

data = pd.read_csv('meteo.csv')
csv = data.to_csv()
put_object = bucket.put_object(Key="meteo.csv", Body=csv)

In [60]:
data = pd.read_csv('hotel_booking.csv',sep="\s{3,}", engine='python')
csv = data.to_csv()
put_object = bucket.put_object(Key="hotel_bookink.csv", Body=csv)

In [61]:
data = pd.read_csv('avg_meteo.csv')
csv = data.to_csv()
put_object = bucket.put_object(Key="avg_meteo.csv", Body=csv)

maintenant nous regardons notre S3 pour s'assurer que nos csv ont bien été chargés  

![title](s3bucket.png)

Il est aussi possible de faire des requêtes SQL ou nous obtenons les mêmes données que dans notre s3 mais les requêtes restent limitées

![title](requetesql.png)

![title](resultsql.png)

## Data Warehouse 

Pour les équipes d'analyse de données, il est préferable de créer une base de données SQL à l'aide de AWS RDS afin qu'elles puissent effectuer des requêtes sur des données propres.

In [25]:
!pip install psycopg2-binary -q
from sqlalchemy import create_engine

engine = create_engine("postgresql+psycopg2://********:*********@************/postgres", echo=True)

In [33]:
meteo = pd.read_csv('meteo.csv')

In [27]:
meteo.to_sql('meteo_7days_35villes', engine)

2021-10-05 10:07:43,526 INFO sqlalchemy.engine.base.Engine select version()
2021-10-05 10:07:43,527 INFO sqlalchemy.engine.base.Engine {}
2021-10-05 10:07:43,541 INFO sqlalchemy.engine.base.Engine select current_schema()
2021-10-05 10:07:43,542 INFO sqlalchemy.engine.base.Engine {}
2021-10-05 10:07:43,555 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-10-05 10:07:43,556 INFO sqlalchemy.engine.base.Engine {}
2021-10-05 10:07:43,563 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-10-05 10:07:43,563 INFO sqlalchemy.engine.base.Engine {}
2021-10-05 10:07:43,570 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2021-10-05 10:07:43,571 INFO sqlalchemy.engine.base.Engine {}
2021-10-05 10:07:43,584 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

![title](pgadmin.png)

Nous pouvons maintenant effectuer des requêtes SQL sur notre base de données afin d'analyser nos données  